In [1]:
import pandas as pd

data = pd.read_csv("/mnt/c/Users/POOYA/Desktop/data.csv")

data

,stationCode,Year_AB_1,Year_AB_2,Month_AB,Day_AB,BARAN_6,BARF_6,AB_BARF_6,BARAN_18,BARF_18,AB_BARF_18
0,110161006,1381,82,10,2,0.0,0.0,25.0,0.0,0.0,0.0
1,110161006,1381,82,10,28,0.0,0.0,9.0,0.0,0.0,0.0
2,110155016,1381,82,1,4,0.0,0.0,0.0,9.0,0.0,0.0
3,110155016,1381,82,1,26,7.0,0.0,0.0,0.0,0.0,0.0
4,110155016,1381,82,1,29,0.0,0.0,0.0,15.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
17811,110164056,1397,98,3,9,3.0,0.0,0.0,0.0,0.0,0.0
17812,110164056,1397,98,3,10,0.0,0.0,0.0,3.0,0.0,0.0
17813,110164056,1397,98,3,11,3.0,0.0,0.0,0.0,0.0,0.0
17814,110164056,1397,98,3,12,1.0,0.0,0.0,0.0,0.0,0.0


In [2]:

def convertAB(Year_AB_1, Year_AB_2, Month_AB, Day_AB, Houre, Minute, Second):
    if Month_AB <=6 and Month_AB >=1:
        if Year_AB_2 >= 20:
            Year = Year_AB_2 + 1300
        else:
            Year = Year_AB_2 + 1400

        Month = Month_AB
        Day = Day_AB
    elif Month_AB <=12 and Month_AB >=7:
        Year = Year_AB_1
        Month = Month_AB
        Day = Day_AB
    else:
        Year = None
        Month = None
        Day = None
    return [int(Year), int(Month), int(Day), int(Houre), int(Minute), int(Second)]



b6 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_6', 'BARF_6', 'AB_BARF_6']]
b6['Houre'] = 6
b6['Minute'] = 30
b6['Second'] = 0
b6['JAM_BARAN'] = b6['BARAN_6'] + b6['AB_BARF_6']

b6[['Date']] = b6.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b6 = b6[['stationCode', 'Date', 'BARAN_6', 'BARF_6', 'AB_BARF_6', 'JAM_BARAN']]
b6.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']



b18 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_18', 'BARF_18', 'AB_BARF_18']]
b18['Houre'] = 18
b18['Minute'] = 30
b18['Second'] = 0
b18['JAM_BARAN'] = b18['BARAN_18'] + b18['AB_BARF_18']

b18[['Date']] = b18.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b18 = b18[['stationCode', 'Date', 'BARAN_18', 'BARF_18', 'AB_BARF_18', 'JAM_BARAN']]
b18.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']




data = pd.concat([b6, b18])
data[['YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND']] = pd.DataFrame(data.DATE.tolist(), index= data.index)
data['DATE'] = data['YEAR'].astype(str) + "-" + data['MONTH'].astype(str).str.zfill(2) + "-" + data['DAY'].astype(str).str.zfill(2) + " " + data['HOURE'].astype(str).str.zfill(2) + ":" + data['MINUTE'].astype(str).str.zfill(2) + ":" + data['SECOND'].astype(str).str.zfill(2) 





data = data[['stationCode', 'DATE', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']]


data.sort_values(by=['stationCode', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND'], inplace=True)



<ipython-input-2-0a8cc1af3b11>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b6['Houre'] = 6
<ipython-input-2-0a8cc1af3b11>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b6['Minute'] = 30
<ipython-input-2-0a8cc1af3b11>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [3]:
data

,stationCode,DATE,YEAR,MONTH,DAY,HOURE,MINUTE,SECOND,BARAN,BARF,AB_BARF,JAM_BARAN
10957,110111003,1346-07-02 06:30:00,1346,7,2,6,30,0,0.0,0.0,0.0,0.0
10957,110111003,1346-07-02 18:30:00,1346,7,2,18,30,0,3.0,0.0,0.0,3.0
10974,110111003,1346-07-09 06:30:00,1346,7,9,6,30,0,0.0,0.0,0.0,0.0
10974,110111003,1346-07-09 18:30:00,1346,7,9,18,30,0,1.0,0.0,0.0,1.0
10977,110111003,1346-07-10 06:30:00,1346,7,10,6,30,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12499,110168010,1387-03-09 18:30:00,1387,3,9,18,30,0,2.0,0.0,0.0,2.0
12492,110168010,1387-04-02 06:30:00,1387,4,2,6,30,0,0.0,0.0,0.0,0.0
12492,110168010,1387-04-02 18:30:00,1387,4,2,18,30,0,2.0,0.0,0.0,2.0
12497,110168010,1387-04-05 06:30:00,1387,4,5,6,30,0,0.0,0.0,0.0,0.0


In [4]:
data.to_csv("data.csv", index=False)

In [ ]:
import pandas as pd
import sqlite3
db_path_precipitation = 'App/dashApp/precipitation/precipitation.sqlite'
db_precipitation = sqlite3.connect(db_path_precipitation, check_same_thread=False)





date = "1398/3/1"
time = "18:30:00"
data = [
    {'stationName': 'فریمان', 'baran': 0, 'barf': 0, 'ab_barf': 0}, 
    {'stationName': 'امور آب سبزوار', 'baran': 0, 'barf': 0, 'ab_barf': 0}, 
    {'stationName': 'چناران', 'baran': 0, 'barf': 0, 'ab_barf': 0},
]

In [ ]:
        exist_precipitation_data = pd.read_sql_query(sql="SELECT * FROM precipitation", con=db_precipitation)
        exist_precipitation_data_columns = list(exist_precipitation_data.columns)
        exist_precipitation_data['DATE'] = exist_precipitation_data['YEAR'].astype(str) + "-" + exist_precipitation_data['MONTH'].astype(str).str.zfill(2) + "-" + exist_precipitation_data['DAY'].astype(str).str.zfill(2) + " " + exist_precipitation_data['HOURE'].astype(str).str.zfill(2) + ":" + exist_precipitation_data['MINUTE'].astype(str).str.zfill(2) + ":" + exist_precipitation_data['SECOND'].astype(str).str.zfill(2)
        exist_precipitation_data["uniqueCode"] = exist_precipitation_data["stationCode"].astype(str) + "-" + exist_precipitation_data["DATE"]

In [ ]:
exist_station = pd.read_sql_query(sql="SELECT * FROM station", con=db_precipitation)

In [ ]:
exist_station

In [ ]:
df = pd.DataFrame(columns=exist_precipitation_data_columns[1:] + ["stationName"])

date = date.split("/")
time = time.split(":")

for i in range(len(data)):
    df.loc[i,"stationCode"] = exist_station[exist_station['stationName'] == data[i]["stationName"]].iloc[0,1]
    df.loc[i,"YEAR"] = date[0]
    df.loc[i,"MONTH"] = date[1].zfill(2)
    df.loc[i,"DAY"] = date[2].zfill(2)
    df.loc[i,"HOURE"] = time[0].zfill(2)
    df.loc[i,"MINUTE"] = time[1].zfill(2)
    df.loc[i,"SECOND"] = time[2].zfill(2)
    df.loc[i,"BARAN"] = data[i]["baran"]
    df.loc[i,"BARF"] = data[i]["barf"]
    df.loc[i,"AB_BARF"] = data[i]["ab_barf"]
    df.loc[i,"JAM_BARAN"] = data[i]["baran"] + data[i]["ab_barf"]
    df.loc[i,"stationName"] = data[i]["stationName"]

df['DATE'] = df['YEAR'].astype(str) + "-" + df['MONTH'].astype(str) + "-" + df['DAY'].astype(str) + " " + df['HOURE'].astype(str) + ":" + df['MINUTE'].astype(str) + ":" + df['SECOND'].astype(str)

df["uniqueCode"] = df["stationCode"].astype(str) + "-" + df["DATE"]


In [ ]:
df

In [ ]:
df.loc[df.duplicated(subset=["stationName"]),:]["stationName"].unique().tolist()

In [ ]:
df["uniqueCode"].isin(exist_precipitation_data["uniqueCode"]).any()

In [ ]:
df.loc[df["uniqueCode"].isin(exist_precipitation_data["uniqueCode"]), :]

In [ ]:
from collections import Counter
a = list((Counter(df["uniqueCode"]) - Counter(exist_precipitation_data["uniqueCode"])).elements())
a

In [ ]:
list(set(list(df["uniqueCode"])) - set(a))

In [ ]:
a

In [ ]:
list(df["uniqueCode"])

In [ ]:
df

In [ ]:
df.drop(['uniqueCode', 'DATE', 'stationName'], axis=1)

In [ ]:
exist_precipitation_data